In [1]:
# import all the libraries we need
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import Key as key

In [2]:
# func to automate the process of getting through the api 
# get the information and immediately convert it into a json file
def get_all_info_brew(url, api_key, start_page=1, page_lim=100): 
    '''takes in a url in the form of a string
       takes in an api_key as a string
       takes optional parameter start_page as an int which defaults to 1
       takes optional parameter page_limit as an int which defaults to 100
       returns a list of jsons files'''
    data = [] 
    p = start_page
    while p <= page_lim:
        url_params = {
                        'key': api_key,
                        "withBreweries" : "Y",
                        "p": p
                    }
        # a list of json files
        data.append(requests.get(url,params=url_params).json())
        p += 1
    return data

In [9]:
url = "https://sandbox-api.brewerydb.com/v2/beers/"
brew_json_data = get_all_info_brew(url, key.key, 1, 200)

[{'currentPage': 2,
  'numberOfPages': 23,
  'totalResults': 1109,
  'data': [{'id': 'hYaduh',
    'name': 'Alt Route - Beer Camp Across America',
    'nameDisplay': 'Alt Route - Beer Camp Across America (2014)',
    'description': 'Brewers are notorious for refusing to take the easy path. Instead, we lead with our hearts and pursue our passions no matter how difficult the road ahead. Alt Route is a tribute to this stubborn spirit that drove us to dodge the status quo and found breweries. Together we chased the siren song of beer with this revamped take on an antique beer style.\r\n\r\nLike us, Victory is one of the few brewers with an allegiance to whole-cone hops. They do extensive work with hop flavor, especially focusing on clever and delicious single-hop beers highlighting the same hop grown in different regions. They too share our roots in home brewing and never lost their passion for working with new beers and styles solely for the joy of making something new.',
    'abv': '6.6'

In [ ]:
# store our raw json data
with open('brew_data_pages_201_to_400_backup.json', 'w') as outfile:
    json.dump(brew_json_pages_data, outfile)

In [11]:
#opens our json file that we saved

with open("brew_data_pages_1_to_200_backup.json") as datajson:
    brew_data =json.load(datajson)

In [22]:
#test if it works
brew_data[0]['data']

[{'id': 'StdoHr',
  'name': '!',
  'nameDisplay': '!',
  'description': 'Imperial Bitter',
  'abv': '8.2',
  'availableId': 3,
  'styleId': 13,
  'isOrganic': 'N',
  'isRetired': 'Y',
  'status': 'verified',
  'statusDisplay': 'Verified',
  'createDate': '2018-05-10 19:07:18',
  'updateDate': '2018-11-02 02:13:07',
  'available': {'id': 3,
   'name': 'Not Available',
   'description': 'Beer is not currently available.'},
  'style': {'id': 13,
   'categoryId': 1,
   'category': {'id': 1,
    'name': 'British Origin Ales',
    'createDate': '2012-03-21 20:06:45'},
   'name': 'Old Ale',
   'shortName': 'Old Ale',
   'description': 'Dark amber to brown in color, old ales are medium to full bodied with a malty sweetness. Hop aroma should be minimal and flavor can vary from none to medium in character intensity. Fruity-ester flavors and aromas can contribute to the character of this ale. Bitterness should be minimal but evident and balanced with malt and/or caramel-like sweetness. Alcohol ty

In [ ]:
# structure of the main json file of properties we might be interested in

{
    "name" : str,
    "id" : str,
    "abv" : float(str),
    "isOrgnanic" : str,
    "isRetired" : str,
    "status" : str,
    "createDate" : str,
    "style" : dict, # look below
    "breweries" : list(dict) # look below category dict section
    
}

# structure of the style dict
{
    "id" : int,
    'category' : dict, # look below
    "name" : str,
    "shortName" : str,
    'ibuMin' : int(str),
    'ibuMax' : int(str),
    'abvMin' : int(str),
    'abvMax' : int(str),
    'srmMin' : int(str),
    'srmMax' : int(str),
    'ogMin' : int(str),
    'fgMin' : int(str),
    'fgMax' : int(str),
    
}
# structure of category dict
{
    "id" : int,
    "name" : str,
}

# brewery structure
{
    "id" : int,
    "name" : str,
    "established" : str,
    "isVerified" : str,
    "locations" : list(dict) # see below
}

# location structure, iterate because of list
{
    "id" : str,
    "name" : str,
    "streetAddress" : str,
    'locality' : str,
    'region' : str,
    'postal' : str,
    "latitude" : float,
    "longitude" : float,
    "locationType" : str,
    "createDate" : str,
}

In [ ]:
# after doing some thinking through and running some samples we decided upon this structure
# what we want the results to be structured as a list of dicts 

[
    {
    "beer_name" : str,
    "beer_id" : str,
    "abv" : float(str),
    "isOrgnanic" : str,
    "isRetired" : str,
    "status" : str,
    "createDate" : str,
    "style_id" : int,
    "style_name" : str,
    "category_id" : int,
    "category_name" : str,
    "style_shortName" : str,
    'style_ibuMin' : int(str),
    'style_ibuMax' : int(str),
    'style_abvMin' : int(str),
    'style_abvMax' : int(str),
    'style_srmMin' : int(str),
    'style_srmMax' : int(str),
    'style_ogMin' : int(str),
    'style_fgMin' : int(str),
    'style_fgMax' : int(str),
    "brewery_id" : int,
    "brewery_name" : str,
    "brewery_established" : str,
    "brewery_isVerified" : str,
    "brewery_locations_id" : str,
    "brewery_locations_name" : str,
    "brewery_locations_map" : [float, float] # [latidude, longitude],
    "brewery_locations_locality" : str
    }
]


In [92]:
# a parser function to clean the data
def parse(data_lst):
    # elements of the style keys list to look for when we parse through style 
    style_lst = ["id", "name", "shortName",'abvMax','srmMin', 'srmMax', "ibuMax", "ibuMin",
                 'ogMin','fgMin','fgMax' ]
    api_dict_list = []
    # read_through each in the data_list
    for index in range(len(data_lst)):
        for api_call in data_lst[index]["data"]:
            api_dict = {}
            api_dict["name"] = api_call["name"]
            if 'abv' in api_call:
                api_dict["abv"] = float(api_call["abv"])
            api_dict["organic?"] = api_call["isOrganic"]
            api_dict["retired?"] = api_call["isRetired"]
            api_dict["status"] = api_call["status"]
            api_dict["createDate"] = api_call["createDate"]
            if "style" in api_call:
                for style in style_lst:
                    if style in api_call["style"]:
                        api_dict[f"{style}"] = api_call["style"][style]
                api_dict["category_name"] = api_call["style"]["category"]["name"]
                api_dict["category_id"] = api_call["style"]["category"]["id"]
            brewery = api_call["breweries"][0]
            api_dict[f"brewery_name"] =  brewery["name"]
            api_dict[f"brewery_id"] = brewery["id"]
            if "established" in api_call["breweries"]:
                api_dict[f"established"] = brewery["established"]
            api_dict["brewery_locations_map"] = []
            api_dict["brewery_locations"] = []
            api_dict["brewery_locality"] = ""
            count = 1
            for idx in range(len(brewery["locations"])):
                brew_location = brewery["locations"][idx]
                api_dict["brewery_locations"].append(brew_location["name"])
                if "latitude" in brew_location and "longitude" in brew_location:
                    api_dict["brewery_locations_map"].append([float(brew_location["latitude"]),
                                                             float(brew_location["longitude"])])
                if "locality" in brew_location:
                    api_dict["brewery_locality"]+= f'{count}. {brew_location["locality"]}' + " "
                    count += 1
            api_dict["number_of_locations"] = len(api_dict["brewery_locations"])
            api_dict_list.append(api_dict)
    return api_dict_list
parsed_data = parse(brew_data)

In [93]:
# turn parsed data into a panda dataframe
brewery_df = pd.DataFrame(parsed_data)

In [94]:
brewery_df

,abv,abvMax,brewery_id,brewery_locality,brewery_locations,brewery_locations_map,brewery_name,category_id,category_name,createDate,...,id,name,number_of_locations,ogMin,organic?,retired?,shortName,srmMax,srmMin,status
0,8.20,9,AAxJnJ,1. Örebro,[Närke Kulturbryggeri],"[[59.2827084, 15.2328471]]",Närke Kulturbryggeri,1,British Origin Ales,2018-05-10 19:07:18,...,13,Old Ale,1,1.058,N,Y,Old Ale,30,12,verified
1,11.10,12,qa1QZU,1. Wallingford,[Main Brewery],"[[39.8909454, -75.3629687]]",Ship Bottom Brewery,3,North American Origin Ales,2013-01-20 21:38:15,...,43,American-Style Imperial Stout,1,1.08,N,N,American Imperial Stout,40,40,verified
2,6.33,5.6,DBkLHq,1. Boothbay,[Main Brewery],"[[43.8807856, -69.6331261]]","Boothbay Craft Brewery, Inc",3,North American Origin Ales,2015-04-20 21:43:59,...,25,American-Style Pale Ale,1,1.044,N,N,American Pale,14,6,verified
3,7.00,9,BU4IJP,1. Milwaukee 2. Milwaukee 3. Grafton,"[Main Brewery, Milwaukee Ale House, Milwaukee ...","[[43.024914, -87.9130348], [43.032936, -87.909...",Milwaukee Brewing Company,9,Other Lager,2012-09-09 13:35:27,...,104,Baltic-Style Porter,3,1.072,N,N,Baltic Porter,40,40,verified
4,5.40,6.5,pvE2w5,1. Geneva,[Main Brewery],"[[42.8332538, -76.979864]]",GAEL Brewing Company,1,British Origin Ales,2015-11-02 20:42:02,...,19,Robust Porter,1,1.045,N,N,Robust Porter,30,30,verified
5,4.80,NaN,FfMi8U,1. Denver,[Main Brewery],"[[39.747316, -105.024345]]",Little Machine Beer,3,North American Origin Ales,2015-11-03 17:04:44,...,40,American-Style Sour Ale,1,NaN,N,N,Sour,NaN,NaN,verified
6,4.60,5,cZDnM8,1. La Verne,[Main Brewery],"[[34.095248, -117.7719245]]",La Verne Brewing Co.,3,North American Origin Ales,2013-04-28 18:27:33,...,36,Golden or Blonde Ale,1,1.045,N,N,Blonde,7,3,verified
7,8.50,6.4,FfMi8U,1. Denver,[Main Brewery],"[[39.747316, -105.024345]]",Little Machine Beer,3,North American Origin Ales,2015-11-03 17:02:33,...,37,American-Style Brown Ale,1,1.04,N,N,American Brown,26,15,verified
8,5.80,5.6,mMnrjZ,1. Albuquerque,[Main Brewery],"[[35.1058608, -106.651184]]",Bow & Arrow Brewing Co,3,North American Origin Ales,2016-05-11 15:05:00,...,25,American-Style Pale Ale,1,1.044,N,N,American Pale,14,6,verified
9,7.60,7.5,H8jawh,1. Livermore,[Main Brewery],"[[37.697526, -121.725189]]",Working Man Brewing Company,3,North American Origin Ales,2013-07-27 14:04:42,...,41,American-Style Black Ale,1,1.056,N,N,Black Ale,35,35,verified


In [96]:
bdf = brewery_df[brewery_df["brewery_locality"].str.contains("New York")].sort_values("name")
bdf = bdf.set_index("brewery_name")
bdf = bdf[["name","brewery_locality"]]
bdf

,name,brewery_locality
brewery_name,,
508 Gastrobrewery,American-Style Imperial Stout,1. New York
508 Gastrobrewery,American-Style India Pale Ale,1. New York
508 Gastrobrewery,American-Style India Pale Ale,1. New York
77 Beer Co,American-Style India Pale Ale,1. New York
77 Beer Co,American-Style Pilsener,1. New York
Third Rail Beer,American-Style Stout,1. New York
508 Gastrobrewery,Belgian-Style Dark Strong Ale,1. New York
508 Gastrobrewery,Belgian-Style Tripel,1. New York
508 Gastrobrewery,Belgian-Style White (or Wit) / Belgian-Style W...,1. New York


In [97]:

# some function to search the database by
def search_by_city(city):
    print("Properties: abv, name, organic?, retired?, ibuMax, abvMax")
    prop = input("Which property do you want to search by?\n")
    bdf = brewery_df[brewery_df["brewery_locality"].str.contains(city)].sort_values(prop)
    bdf = bdf.set_index("brewery_name")
    bdf = bdf[["brewery_locality", prop]]
    return bdf 

In [100]:
search_by_city("New York")

Properties: abv, name, organic?, retired?, ibuMax, abvMax
Which property do you want to search by?
retired?


,brewery_locality,retired?
brewery_name,,
508 Gastrobrewery,1. New York,N
Original Sin,1. New York,N
Third Rail Beer,1. New York,N
77 Beer Co,1. New York,N
77 Beer Co,1. New York,N
508 Gastrobrewery,1. New York,N
508 Gastrobrewery,1. New York,N
508 Gastrobrewery,1. New York,N
508 Gastrobrewery,1. New York,N
